In [2]:
#appending fourth image block to galfit output mosaic (contains residual with same flux stretch as that of
#the WISE signal -- data -- image)
import os
import numpy as np
from astropy.io import fits
from astropy.visualization import simple_norm
from astropy.wcs import WCS
from astropy.table import Table
from astropy.io import ascii
from matplotlib import pyplot as plt
%matplotlib inline
from scipy.stats import scoreatpercentile
import glob

homedir = os.getenv("HOME")

In [19]:
#vfcut.fits for SNR>10 subsample, sga_cut.fits for PA,BA fixed

vf = Table.read(homedir+'/vfcut.fits',format='ascii')
dummycat = Table.read(homedir+'/dummycat.fits',format='ascii')

In [11]:
def four_panel_mosaic(index,sample=vf,dummycat=dummycat):
    
    galname = sample['VFID'][index]
    objname = sample['prefix'][index]
    #output_fits for vf, output_fits_fixed for sga
    filename = glob.glob(homedir+'/output_fits/'+galname+'*.fits')[0]
    
    print(index)
    ind = np.where(sample['VFID'] == galname)[0]
    ind = int(ind)
    
    if vf['VFID'][ind] in dummycat['central galaxy']:
        ncomp = len(np.where(dummycat['central galaxy'] == galname)[0])
        pngname = objname+'-unwise-w3-{}Comp-galfit-out-conv.png'.format(ncomp+1)
    else:
        pngname = objname+'-unwise-w3-1Comp-galfit-out-conv.png'
    
    
    image,h = fits.getdata(filename,1,header=True)
    model = fits.getdata(filename,2)
    residual = fits.getdata(filename,3)

    wcs = WCS(h)

    images = [image,model,residual,residual]
    titles = ['image','model','residual (img stretch)','residual (res stretch)']

    percentile1=.5
    percentile2=99.5
    p1residual=5
    p2residual=99

    v1 = [scoreatpercentile(image,percentile1),
        scoreatpercentile(image,percentile1),
        scoreatpercentile(image,percentile1),
        scoreatpercentile(residual,p1residual)]

    v2 = [scoreatpercentile(image,percentile2),
        scoreatpercentile(image,percentile2),        
        scoreatpercentile(image,percentile2),
        scoreatpercentile(residual,p2residual)]

    norms = [simple_norm(image,'asinh',max_percent=percentile2),
            simple_norm(image,'asinh',max_percent=percentile2),
            simple_norm(image,'asinh',max_percent=percentile2),
            simple_norm(residual,'linear',max_percent=p2residual)]

    plt.figure(figsize=(14,6))
    plt.subplots_adjust(wspace=.0)
    for i,im in enumerate(images):
        plt.subplot(1,4,i+1,projection=wcs)
        plt.imshow(im,origin='lower',vmin=v1[i],vmax=v2[i],norm=norms[i])
        plt.xlabel('RA')
        if i == 0:
            plt.ylabel('DEC')
        else:
            plt.ylabel(' ')
            ax = plt.gca()
            ax.set_yticks([])     
        plt.title(titles[i],fontsize=16)
    
    
    plt.savefig(pngname)
    plt.close()

In [ ]:
#output_mosaics for vf, output_mosaics_fixed for sga
os.chdir(homedir+'/output_mosaics')

for index in range(470,len(vf)):
    four_panel_mosaic(index)

In [8]:
np.where(vf['VFID']=='VFID2171')

(array([412]),)

In [ ]:
#PA,BA fixed failure indices: 412,464
#PA,BA free failure indices: 238,321,456,469

#719